In [1]:
import streamlit as st
from streamlit_option_menu import option_menu
import easyocr
from PIL import Image
import pandas as pd
from bidi import get_display
import numpy as np
import re
import io
import sqlite3

2024-11-27 12:49:44.407 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
def image_to_text(path):
  #Reading image
  input_img = Image.open(path)

  #Converting image to array format
  image_array = np.array(input_img)

  reader = easyocr.Reader(['en'])
  text = reader.readtext(image_array, detail=0)

  return text, input_img


In [3]:
text_img, input_img = image_to_text("E:/DATA SCIENCE - COURSE - GUVI/GUVI Project/BizCardX Extracting Business Card Data with OCR/1.png")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [4]:
def Extracted_text(texts):
  ext_dict = {"NAME":[], "DESIGNATION":[], "COMPANY_NAME":[], "CONTACT":[], "EMAIL":[], "WEBSITE":[],"ADDRESS":[], "PINCODE":[],}

  ext_dict["NAME"].append(texts[0])
  ext_dict["DESIGNATION"].append(texts[1])

  for i in range(2, len(texts)):

    if texts[i].startswith("+") or (texts[i].replace("-","").isdigit() and '-' in texts[i]):
      ext_dict["CONTACT"].append(texts[i])

    elif "@" in texts[i] and ".com" in texts[i]:
      ext_dict["EMAIL"].append(texts[i])

    elif "www" in texts[i] or "WWW" in texts[i] or "Www" in texts[i] or "wWw" in texts[i] or "wwW" in texts[i]:
      small = texts[i].lower()
      ext_dict["WEBSITE"].append(small)

    elif "Tamil Nadu" in texts[i] or "TamilNadu" in texts[i] or texts[i].isdigit():
      ext_dict["PINCODE"].append(texts[i])

    elif re.match(r"^[A-Za-z]",texts[i]):
      ext_dict["COMPANY_NAME"].append(texts[i])

    else:
      remove_colon = re.sub(r'[,;]','',texts[i])
      ext_dict["ADDRESS"].append(remove_colon)

  for key,value in ext_dict.items():
    if len(value)>0:
      concadenate = " ".join(value)
      ext_dict[key] = [concadenate]

    else:
      value = "NA"
      ext_dict[key] = [value]

  return ext_dict

In [5]:
text_ext_data = Extracted_text(text_img)

In [6]:
df = pd.DataFrame(text_ext_data)
df

,NAME,DESIGNATION,COMPANY_NAME,CONTACT,EMAIL,WEBSITE,ADDRESS,PINCODE
0,Selva,DATA MANAGER,selva digitals,+123-456-7890 +123-456-7891,hello@XYZ1.com,www xyzi.com,123 ABC St Chennai,TamilNadu 600113


In [7]:
#Converting Image to Byte

Img_byte = io.BytesIO()
input_img.save(Img_byte, format='PNG')
Img_data = Img_byte.getvalue()

#Creating Dictionary

card = {"IMAGE":[Img_data]}
df_1 = pd.DataFrame(card)
Concat_df = pd.concat([df,df_1], axis=1)
Concat_df

,NAME,DESIGNATION,COMPANY_NAME,CONTACT,EMAIL,WEBSITE,ADDRESS,PINCODE,IMAGE
0,Selva,DATA MANAGER,selva digitals,+123-456-7890 +123-456-7891,hello@XYZ1.com,www xyzi.com,123 ABC St Chennai,TamilNadu 600113,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...


In [8]:
mydb = sqlite3.connect("Bizcard_data.db")
cur = mydb.cursor()

#Table Creation

Create_tab_query = """CREATE TABLE IF NOT EXISTS bizcard_details(name varchar(250),
                                                                 designation varchar(250),
                                                                 company_name varchar(250),
                                                                 contact varchar(250),
                                                                 email varchar(250),
                                                                 website text,
                                                                 address text,
                                                                 pincode varchar(250),
                                                                 image text)"""

cur.execute(Create_tab_query)
mydb.commit()

In [9]:
# Insert Values

Insert_val_query = """INSERT INTO bizcard_details(name,
                                                  designation,
                                                  company_name,
                                                  contact,
                                                  email,
                                                  website,
                                                  address,
                                                  pincode,
                                                  image)
                      VALUES(?,?,?,?,?,?,?,?,?)"""

datas = Concat_df.values.tolist()
cur.executemany(Insert_val_query, datas)
mydb.commit()

In [10]:
#Select Query
mydb = sqlite3.connect("Bizcard_data.db")
cur = mydb.cursor()

Select_query = """SELECT NAME FROM bizcard_details"""
cur.execute(Select_query)
table1 = cur.fetchall()
mydb.commit()

names=[]

for i in table1:
  names.append(i[0])

names


['Selva']

In [ ]:
text_img, input_img = image_to_text("/content/1.png")


In [ ]:
text_img, input_img = image_to_text("/content/2.png")


In [ ]:
text_img, input_img = image_to_text("/content/3.png")


In [ ]:
text_img, input_img = image_to_text("/content/4.png")
